In [1]:
!pip install -q torch==1.10.1 torchvision 

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torch.nn.parameter import Parameter
import torchvision
import numpy as np
from SpykeTorch import snn
from SpykeTorch import functional as sf
from SpykeTorch import visualization as vis
from SpykeTorch import utils
from torchvision import transforms

#use_cuda = True
use_cuda = False

In [5]:
class FlyEye(nn.Module):
    def __init__(self):
        super(FlyEye, self).__init__()

        # photoreceptor cells
        #self.R1_6 = snn.Convolution(1,1,1)
        self.convR7 = snn.Convolution(6,1,1)
        self.convR8 = snn.Convolution(1,1,1)
        
        self.convR7_t = 10
        self.convR8_t = 10
        # Lamina
        self.convL1 = snn.Convolution(1,1,1)
        self.convL2 = snn.Convolution(1,1,1)
        self.convL3 = snn.Convolution(1,1,1)
        self.convL4 = snn.Convolution(1,1,1)
        self.convL5 = snn.Convolution(1,1,1)
        
        self.convC2 = snn.Convolution(1,1,1)
        self.convC3 = snn.Convolution(1,1,1)
        
        self.convL1_t = 10
        self.convL2_t = 10
        self.convL3_t = 10
        self.convL4_t = 10
        self.convL5_t = 10
        self.convC2_t = 10
        self.convC3_t = 10
        
        # Medulla
        self.convMi1 = snn.Convolution(1,1,1)
        self.convMi4 = snn.Convolution(1,1,1)
        self.convMi9 = snn.Convolution(1,1,1)
        self.convMi15 = snn.Convolution(1,1,1)
        
        self.convTm1 = snn.Convolution(1,1,1)
        self.convTm2 = snn.Convolution(1,1,1)
        self.convTm3 = snn.Convolution(1,1,1)
        self.convTm4 = snn.Convolution(1,1,1)
        self.convTm6 = snn.Convolution(1,1,1)
        self.convTm9 = snn.Convolution(1,1,1)
        self.convTm20 = snn.Convolution(1,1,1)
        self.convTmY5a = snn.Convolution(1,1,1)
        
        self.convT2 = snn.Convolution(1,1,1)
        self.convT2a = snn.Convolution(1,1,1)
        self.convT3 = snn.Convolution(1,1,1)
        
        self.convMi1_t = 10
        self.convMi4_t = 10
        self.convMi9_t = 10
        self.convMi15_t = 10
        self.convTm1_t = 10
        self.convTm2_t = 10
        self.convTm3_t = 10
        self.convTm4_t = 10
        self.convTm6_t = 10
        self.convTm9_t = 10
        self.convTm20_t = 10
        self.convTmY5a_t = 10
        self.convTmT2_t = 10
        self.convTmT2a_t = 10
        self.convTmT3_t = 10
        
        # Lobula
        self.convLC4 = snn.Convolution(1,1,1)
        self.convLC17 = snn.Convolution(1,1,1)
        
        self.convLC4_t = 10
        self.convLC17_t = 10
        
        # Cenral brain
        self.convCB = snn.Convolution(1,1,1)
        #self.convCB_t = inf
        
        # STDP applying
        #self.stdpR1_6 = snn.STDP(self.convR1_6, (0.004, -0.003))
        self.stdpR7 = snn.STDP(self.convR7, (0.004, -0.003))
        self.stdpR8 = snn.STDP(self.convR8, (0.004, -0.003))        
        
        self.stdpL1 = snn.STDP(self.convL1, (0.004, -0.003))
        self.stdpL2 = snn.STDP(self.convL2, (0.004, -0.003))
        self.stdpL3 = snn.STDP(self.convL3, (0.004, -0.003))
        self.stdpL4 = snn.STDP(self.convL4, (0.004, -0.003))
        self.stdpL5 = snn.STDP(self.convL5, (0.004, -0.003))
        
        self.stdpC2 = snn.STDP(self.convC2, (0.004, -0.003))
        self.stdpC3 = snn.STDP(self.convC3, (0.004, -0.003))
        
        self.stdpMi1 = snn.STDP(self.convMi1, (0.004, -0.003))
        self.stdpMi14 = snn.STDP(self.convMi4, (0.004, -0.003))
        self.stdpMi9 = snn.STDP(self.convMi9, (0.004, -0.003))
        self.stdpMi15 = snn.STDP(self.convMi15, (0.004, -0.003))
        
        self.stdpTm1 = snn.STDP(self.convTm1, (0.004, -0.003))
        self.stdpTm2 = snn.STDP(self.convTm2, (0.004, -0.003))
        self.stdpTm3 = snn.STDP(self.convTm3, (0.004, -0.003))
        self.stdpTm4 = snn.STDP(self.convTm4, (0.004, -0.003))
        self.stdpTm6 = snn.STDP(self.convTm6, (0.004, -0.003))
        self.stdpTm9 = snn.STDP(self.convTm9, (0.004, -0.003))
        self.stdpTm20 = snn.STDP(self.convTm20, (0.004, -0.003))
        self.stdpTmY5a = snn.STDP(self.convTmY5a, (0.004, -0.003))
        
        self.stdpT2 = snn.STDP(self.convT2, (0.004, -0.003))
        self.stdpT2a = snn.STDP(self.convT2a, (0.004, -0.003))
        self.stdpT3 = snn.STDP(self.convT3, (0.004, -0.003))
        
        self.stdpLC4 = snn.STDP(self.convLC4, (0.004, -0.003))
        self.stdpLC17 = snn.STDP(self.convLC17, (0.004, -0.003))
        
        self.anti_stdp_central_brain = snn.STDP(self.convCB, (-0.004, 0.005), False, 0.2, 0.8)
        self.stdp_central_brain = snn.STDP(self.convCB, (0.004, -0.003), False, 0.2, 0.8)
        
        
        
        
        self.ctx = {"input_spikes":None, "potentials":None, "output_spikes":None, "winners":None}
        self.spk_cnt1 = 0
        self.spk_cnt2 = 0

        
        
    def forward(self, input, max_layer):
        # input - данные после обработки
        
        
        
        
        # R1_6, R7, R8-------------------------------------------------------------------------------------------------
        R1_6 = input

        potR7 = self.convR7(input)
        spkR7, potR7 = sf.fire(potR7, self.convR7_t, True)

        if max_layer == 'R7':
            pot = sf.pointwise_inhibition(potR7)
            spk = pot.sign()
            winners = sf.get_k_winners(pot, spikes=spk)
            self.ctx["input_spikes"] = input
            self.ctx["potentials"] = pot
            self.ctx["output_spikes"] = spk
            self.ctx["winners"] = winners
            return spk, pot
        
        potR8 = self.convR8(input)
        spkR8, potR8 = sf.fire(potR8, self.convR8_t, True)
        
        if max_layer == 'R8':
            pot = sf.pointwise_inhibition(potR8)
            spk = pot.sign()
            winners = sf.get_k_winners(pot, spikes=spk)
            self.ctx["input_spikes"] = input
            self.ctx["potentials"] = pot
            self.ctx["output_spikes"] = spk
            self.ctx["winners"] = winners
            return spk, pot
        
        
        
        
        # L1, L2, L3, L4, L5------------------------------------------------------------------------------------------
        # L1: R1_6 + R8 as input
        potL1 = self.convL1(torch.cat([R1_6,spkR8], dim=1))
        spkL1, potL1 = sf.fire(potL1, self.convL1_t, True)
        if max_layer == 'L1':
            pot = sf.pointwise_inhibition(potL1)
            spk = pot.sign()
            winners = sf.get_k_winners(pot, spikes=spk)
            self.ctx["input_spikes"] = torch.cat([R1_6,spkR8], dim=1)
            self.ctx["potentials"] = pot
            self.ctx["output_spikes"] = spk
            self.ctx["winners"] = winners
            return spk, pot
                                                 
        # L2: R1_6 as input
        potL2 = self.convL2(R1_6)
        spkL2, potL2 = sf.fire(potL2, self.convL2_t, True)
        if max_layer == 'L2':
            pot = sf.pointwise_inhibition(potL2)
            spk = pot.sign()
            winners = sf.get_k_winners(pot, spikes=spk)
            self.ctx["input_spikes"] = R1_6
            self.ctx["potentials"] = pot
            self.ctx["output_spikes"] = spk
            self.ctx["winners"] = winners
            return spk, pot
                                                 
        # L3: R1_6, R7, R8 as input
        potL3 = self.convL3(torch.cat([R1_6,spkR7,spkR8], dim=1))
        spkL3, potL3 = sf.fire(potL3, self.convL3_t, True)
        if max_layer == 'L3':
            pot = sf.pointwise_inhibition(potL3)
            spk = pot.sign()
            winners = sf.get_k_winners(pot, spikes=spk)
            self.ctx["input_spikes"] = torch.cat([R1_6,spkR7,spkR8], dim=1)
            self.ctx["potentials"] = pot
            self.ctx["output_spikes"] = spk
            self.ctx["winners"] = winners
            return spk, pot
                                                 
        # L4: R1_6 as input
        potL4 = self.convL4(R1_6)
        spkL4, potL4 = sf.fire(potL4, self.convL4_t, True)
        if max_layer == 'L4':
            pot = sf.pointwise_inhibition(potL4)
            spk = pot.sign()
            winners = sf.get_k_winners(pot, spikes=spk)
            self.ctx["input_spikes"] = R1_6
            self.ctx["potentials"] = pot
            self.ctx["output_spikes"] = spk
            self.ctx["winners"] = winners
            return spk, pot
        
        # L5: R1_6 + R8 as input
        potL5 = self.convL5(torch.cat([R1_6,spkR8], dim=1))
        spkL5, potL5 = sf.fire(potL5, self.convL5_t, True)
        if max_layer == 'L5':
            pot = sf.pointwise_inhibition(potL5)
            spk = pot.sign()
            winners = sf.get_k_winners(pot, spikes=spk)
            self.ctx["input_spikes"] = torch.cat([R1_6,spkR8], dim=1)
            self.ctx["potentials"] = pot
            self.ctx["output_spikes"] = spk
            self.ctx["winners"] = winners
            return spk, pot                                         
    

        # C2, C3------------------------------------------------------------------------------------------------------
        # C2: L1, L5(5,5) as input
        potC2 = self.convC2(torch.cat([spkL1,
                                       sf.pooling(spkL5, 5, 1, 2)],
                                      dim=1))
        spkC2, potC2 = sf.fire(potC2, self.convC2_t, True)
        if max_layer == 'C2':
            pot = sf.pointwise_inhibition(potC2)
            spk = pot.sign()
            winners = sf.get_k_winners(pot, spikes=spk)
            self.ctx["input_spikes"] = torch.cat([spkL1,
                                       sf.pooling(spkL5, 5, 1, 2)],
                                      dim=1)
            self.ctx["potentials"] = pot
            self.ctx["output_spikes"] = spk
            self.ctx["winners"] = winners
            return spk, pot
        
        # C3: L1, L2, L3, L5(3,3) as input
        potC3 = self.convC3(torch.cat([spkL1, spkL2, spkL3,
                                       sf.pooling(spkL5, 3, 1, 1)],
                                      dim=1))
        spkC3, potC3 = sf.fire(potC3, self.convC3_t, True)
        if max_layer == 'C3':
            pot = sf.pointwise_inhibition(potC3)
            spk = pot.sign()
            winners = sf.get_k_winners(pot, spikes=spk)
            self.ctx["input_spikes"] = torch.cat([spkL1, spkL2, spkL3,
                                       sf.pooling(spkL5, 3, 1, 1)],
                                      dim=1)
            self.ctx["potentials"] = pot
            self.ctx["output_spikes"] = spk
            self.ctx["winners"] = winners
            return spk, pot
        
        
        # Mi1, Mi4, Mi9, Mi15-----------------------------------------------------------------------------------------
        # Mi1: R8, L1, L3, C2, C3, L5(3,3) as input
        potMi1 = self.convMi1(torch.cat([spkR8, spkL1, spkL3,spkC2,spkC3,
                                       sf.pooling(spkL5, 3, 1, 1)],
                                      dim=1))
        spkMi1, potMi1 = sf.fire(potMi1, self.convMi1_t, True)
        if max_layer == 'Mi1':
            pot = sf.pointwise_inhibition(potMi1)
            spk = pot.sign()
            winners = sf.get_k_winners(pot, spikes=spk)
            self.ctx["input_spikes"] = torch.cat([spkR8, spkL1, spkL3,spkC2,spkC3,
                                       sf.pooling(spkL5, 3, 1, 1)],
                                      dim=1)
            self.ctx["potentials"] = pot
            self.ctx["output_spikes"] = spk
            self.ctx["winners"] = winners
            return spk, pot
        
        
        # Mi4: R8, L2, L3, C2, C3, L5(3,3) as input
        potMi4 = self.convMi4(torch.cat([spkR8, spkL2, spkL3,spkC2,spkC3, 
                                       sf.pooling(spkL5, 3, 1, 1)],
                                      dim=1))
        spkMi4, potMi4 = sf.fire(potMi4, self.convMi4_t, True)
        if max_layer == 'Mi4':
            pot = sf.pointwise_inhibition(potMi4)
            spk = pot.sign()
            winners = sf.get_k_winners(pot, spikes=spk)
            self.ctx["input_spikes"] = torch.cat([spkR8, spkL2, spkL3,spkC2,spkC3, 
                                       sf.pooling(spkL5, 3, 1, 1)],
                                      dim=1)
            self.ctx["potentials"] = pot
            self.ctx["output_spikes"] = spk
            self.ctx["winners"] = winners
            return spk, pot
        
        # Mi9: R7, R8, L2, L3(3,3), L4(3,3) as input
        potMi9 = self.convMi9(torch.cat([spkR7, spkR8, spkL2, 
                                       sf.pooling(spkL3, 3, 1, 1),
                                       sf.pooling(spkL4, 3, 1, 1)],
                                      dim=1))
        spkMi9, potMi9 = sf.fire(potMi9, self.convMi9_t, True)
        if max_layer == 'Mi9':
            pot = sf.pointwise_inhibition(potMi9)
            spk = pot.sign()
            winners = sf.get_k_winners(pot, spikes=spk)
            self.ctx["input_spikes"] = torch.cat([spkR7, spkR8, spkL2, 
                                       sf.pooling(spkL3, 3, 1, 1),
                                       sf.pooling(spkL4, 3, 1, 1)],
                                      dim=1)
            self.ctx["potentials"] = pot
            self.ctx["output_spikes"] = spk
            self.ctx["winners"] = winners
            return spk, pot
        
        # Mi15: R8(3,3), L5(3,3) as input
        potMi15 = self.convMi15(torch.cat([ 
                                       sf.pooling(spkR8, 3, 1, 1),
                                       sf.pooling(spkL5, 3, 1, 1)],
                                      dim=1))
        spkMi15, potMi15 = sf.fire(potMi15, self.convMi15_t, True)
        if max_layer == 'Mi15':
            pot = sf.pointwise_inhibition(potMi15)
            spk = pot.sign()
            winners = sf.get_k_winners(pot, spikes=spk)
            self.ctx["input_spikes"] = torch.cat([ 
                                       sf.pooling(spkR8, 3, 1, 1),
                                       sf.pooling(spkL5, 3, 1, 1)],
                                      dim=1)
            self.ctx["potentials"] = pot
            self.ctx["output_spikes"] = spk
            self.ctx["winners"] = winners
            return spk, pot
        
        
        # Tm1, Tm2, Tm3, Tm4, Tm6, Tm9, Tm20, TmY5a------------------------------------------------------------------
        # Tm1: L2, L5, C2, C3, Mi1, Mi4, Mi9
        potTm1 = self.convTm1(torch.cat([spkL2, spkL5, spkC2, spkC3,
                                         spkMi1, spkMi4, spkMi9],
                                      dim=1))
        spkTm1, potTm1 = sf.fire(potTm1, self.convTm1_t, True)
        if max_layer == 'Tm1':
            pot = sf.pointwise_inhibition(potTm1)
            spk = pot.sign()
            winners = sf.get_k_winners(pot, spikes=spk)
            self.ctx["input_spikes"] = torch.cat([spkL2, spkL5, spkC2, spkC3,
                                         spkMi1, spkMi4, spkMi9],
                                      dim=1)
            self.ctx["potentials"] = pot
            self.ctx["output_spikes"] = spk
            self.ctx["winners"] = winners
            return spk, pot
        
        
        # Tm2: L2, L4(3,3), C3, Mi1, Mi9
        potTm2 = self.convTm2(torch.cat([spkL2, spkC3, spkMi1, spkMi9,
                                         sf.pooling(spkL4, 3, 1, 1)],
                                      dim=1))
        spkTm2, potTm2 = sf.fire(potTm2, self.convTm2_t, True)
        if max_layer == 'Tm2':
            pot = sf.pointwise_inhibition(potTm2)
            spk = pot.sign()
            winners = sf.get_k_winners(pot, spikes=spk)
            self.ctx["input_spikes"] = torch.cat([spkL2, spkC3, spkMi1, spkMi9,
                                         sf.pooling(spkL4, 3, 1, 1)],
                                      dim=1)
            self.ctx["potentials"] = pot
            self.ctx["output_spikes"] = spk
            self.ctx["winners"] = winners
            return spk, pot
        
        # Tm3: L3(3,3), C2(3,3), Mi4(3,3), Mi9(3,3), L1(5,5), L5(5,5), Mi1(5,5)
        potTm3 = self.convTm3(torch.cat([
                                         sf.pooling(spkL3, 3, 1, 1),
                                         sf.pooling(spkC2, 3, 1, 1),
                                         sf.pooling(spkMi4, 3, 1, 1),
                                         sf.pooling(spkMi9, 3, 1, 1),                
                                         sf.pooling(spkL1, 5, 1, 2),
                                         sf.pooling(spkL5, 5, 1, 2),
                                         sf.pooling(spkMi1, 5, 1, 2)],
                                      dim=1))
        spkTm3, potTm3 = sf.fire(potTm3, self.convTm3_t, True)
        if max_layer == 'Tm3':
            pot = sf.pointwise_inhibition(potTm3)
            spk = pot.sign()
            winners = sf.get_k_winners(pot, spikes=spk)
            self.ctx["input_spikes"] = torch.cat([
                                         sf.pooling(spkL3, 3, 1, 1),
                                         sf.pooling(spkC2, 3, 1, 1),
                                         sf.pooling(spkMi4, 3, 1, 1),
                                         sf.pooling(spkMi9, 3, 1, 1),                
                                         sf.pooling(spkL1, 5, 1, 2),
                                         sf.pooling(spkL5, 5, 1, 2),
                                         sf.pooling(spkMi1, 5, 1, 2)],
                                      dim=1)
            self.ctx["potentials"] = pot
            self.ctx["output_spikes"] = spk
            self.ctx["winners"] = winners
            return spk, pot
        
        
        # Tm4: L4(3,3), Mi4(3), Mi9(3), L2(5), C3(5)
        potTm4 = self.convTm4(torch.cat([
                                         sf.pooling(spkL4, 3, 1, 1),
                                         sf.pooling(spkMi4, 3, 1, 1),
                                         sf.pooling(spkMi9, 3, 1, 1),                
                                         sf.pooling(spkL2, 5, 1, 2),
                                         sf.pooling(spkC3, 5, 1, 2)],
                                      dim=1))
        spkTm4, potTm4 = sf.fire(potTm4, self.convTm4_t, True)
        if max_layer == 'Tm4':
            pot = sf.pointwise_inhibition(potTm4)
            spk = pot.sign()
            winners = sf.get_k_winners(pot, spikes=spk)
            self.ctx["input_spikes"] = torch.cat([
                                         sf.pooling(spkL4, 3, 1, 1),
                                         sf.pooling(spkMi4, 3, 1, 1),
                                         sf.pooling(spkMi9, 3, 1, 1),                
                                         sf.pooling(spkL2, 5, 1, 2),
                                         sf.pooling(spkC3, 5, 1, 2)],
                                      dim=1)
            self.ctx["potentials"] = pot
            self.ctx["output_spikes"] = spk
            self.ctx["winners"] = winners
            return spk, pot
        
        
        # Tm6: Mi1(3), Mi15(3), L5(5), Mi9(5)
        potTm6 = self.convTm6(torch.cat([sf.pooling(spkMi1, 3, 1, 1),
                                         sf.pooling(spkMi15, 3, 1, 1),               
                                         sf.pooling(spkL5, 5, 1, 2),
                                         sf.pooling(spkMi9, 5, 1, 2)],
                                      dim=1))
        spkTm6, potTm6 = sf.fire(potTm6, self.convTm6_t, True)
        if max_layer == 'Tm6':
            pot = sf.pointwise_inhibition(potTm6)
            spk = pot.sign()
            winners = sf.get_k_winners(pot, spikes=spk)
            self.ctx["input_spikes"] = torch.cat([sf.pooling(spkMi1, 3, 1, 1),
                                         sf.pooling(spkMi15, 3, 1, 1),               
                                         sf.pooling(spkL5, 5, 1, 2),
                                         sf.pooling(spkMi9, 5, 1, 2)],
                                      dim=1)
            self.ctx["potentials"] = pot
            self.ctx["output_spikes"] = spk
            self.ctx["winners"] = winners
            return spk, pot
        
        # Tm9: L4(3), Mi4(3), L2, C2, C3
        potTm9 = self.convTm9(torch.cat([sf.pooling(spkL4, 3, 1, 1),
                                        sf.pooling(spkMi4, 3, 1, 1),
                                        spkL2, spkC2, spkC3],
                                      dim=1))
        spkTm9, potTm9 = sf.fire(potTm9, self.convTm9_t, True)
        if max_layer == 'Tm9':
            pot = sf.pointwise_inhibition(potTm9)
            spk = pot.sign()
            winners = sf.get_k_winners(pot, spikes=spk)
            self.ctx["input_spikes"] = torch.cat([sf.pooling(spkL4, 3, 1, 1),
                                        sf.pooling(spkMi4, 3, 1, 1),
                                        spkL2, spkC2, spkC3],
                                      dim=1)
            self.ctx["potentials"] = pot
            self.ctx["output_spikes"] = spk
            self.ctx["winners"] = winners
            return spk, pot
        # Tm20: Mi4(3), R7, R8, L2, C3, Mi1
        potTm20 = self.convTm20(torch.cat([sf.pooling(spkMi4, 3, 1, 1),
                                        spkR7, spkR8, spkL2, spkC3, spkMi1],
                                      dim=1))
        spkTm20, potTm20 = sf.fire(potTm20, self.convTm20_t, True)
        if max_layer == 'Tm20':
            pot = sf.pointwise_inhibition(potTm20)
            spk = pot.sign()
            winners = sf.get_k_winners(pot, spikes=spk)
            self.ctx["input_spikes"] = torch.cat([sf.pooling(spkMi4, 3, 1, 1),
                                        spkR7, spkR8, spkL2, spkC3, spkMi1],
                                      dim=1)
            self.ctx["potentials"] = pot
            self.ctx["output_spikes"] = spk
            self.ctx["winners"] = winners
            return spk, pot
        
        
        # TmY5a: L5(3), Mi4(3), Mi9(3)
        potTmY5a = self.convTmY5a(torch.cat([sf.pooling(spkL5, 3, 1, 1),
                                            sf.pooling(spkMi4, 3, 1, 1),
                                            sf.pooling(spkMi9, 3, 1, 1)],
                                      dim=1))
        spkTmY5a, potTmY5a = sf.fire(potTmY5a, self.convTmY5a_t, True)
        if max_layer == 'TmY5a':
            pot = sf.pointwise_inhibition(potTmY5a)
            spk = pot.sign()
            winners = sf.get_k_winners(pot, spikes=spk)
            self.ctx["input_spikes"] = torch.cat([sf.pooling(spkL5, 3, 1, 1),
                                            sf.pooling(spkMi4, 3, 1, 1),
                                            sf.pooling(spkMi9, 3, 1, 1)],
                                      dim=1)
            self.ctx["potentials"] = pot
            self.ctx["output_spikes"] = spk
            self.ctx["winners"] = winners
            return spk, pot
        

        # T2, T2a, T3-----------------------------------------------------------------------------------------------
        # T2: Mi1(3), Tm1(3), Tm3(3), Tm4(3), TmY5a(3)
        potT2 = self.convT2(torch.cat([sf.pooling(spkMi1, 3, 1, 1),
                                            sf.pooling(spkTm1, 3, 1, 1),
                                            sf.pooling(spkTm3, 3, 1, 1),
                                            sf.pooling(spkTm4, 3, 1, 1),
                                            sf.pooling(spkTmY5a, 3, 1, 1)],
                                      dim=1))
        spkT2, potT2 = sf.fire(potT2, self.convT2_t, True)
        if max_layer == 'T2':
            pot = sf.pointwise_inhibition(potT2)
            spk = pot.sign()
            winners = sf.get_k_winners(pot, spikes=spk)
            self.ctx["input_spikes"] = torch.cat([sf.pooling(spkMi1, 3, 1, 1),
                                            sf.pooling(spkTm1, 3, 1, 1),
                                            sf.pooling(spkTm3, 3, 1, 1),
                                            sf.pooling(spkTm4, 3, 1, 1),
                                            sf.pooling(spkTmY5a, 3, 1, 1)],
                                      dim=1)
            self.ctx["potentials"] = pot
            self.ctx["output_spikes"] = spk
            self.ctx["winners"] = winners
            return spk, pot
        
        
        # T2a: L2, L5, C2, C3, Mi4, Tm1, Tm2
        potT2a = self.convT2a(torch.cat([spkL2, spkL5, spkC2, spkC3, spkMi4, spkTm1, spkTm2], dim=1))
        spkT2a, potT2a = sf.fire(potT2a, self.convT2a_t, True)
        if max_layer == 'T2a':
            pot = sf.pointwise_inhibition(potT2a)
            spk = pot.sign()
            winners = sf.get_k_winners(pot, spikes=spk)
            self.ctx["input_spikes"] = torch.cat([spkL2, spkL5, spkC2, spkC3, spkMi4, spkTm1, spkTm2], dim=1)
            self.ctx["potentials"] = pot
            self.ctx["output_spikes"] = spk
            self.ctx["winners"] = winners
            return spk, pot
        
        
        # T3: L2, L4, L5, C2, C3, Mi1, Mi9, Tm1, Tm2, Tm3, Tm6, TmY5a - 3 window
        potT3 = self.convT3(torch.cat([sf.pooling(spkL2, 3, 1, 1),
                                            sf.pooling(spkL4, 3, 1, 1),
                                            sf.pooling(spkL5, 3, 1, 1),
                                            sf.pooling(spkC2, 3, 1, 1),
                                            sf.pooling(spkC3, 3, 1, 1),
                                            sf.pooling(spkMi1, 3, 1, 1),
                                            sf.pooling(spkMi9, 3, 1, 1),
                                            sf.pooling(spkTm1, 3, 1, 1),
                                            sf.pooling(spkTm2, 3, 1, 1),
                                            sf.pooling(spkTm3, 3, 1, 1),
                                            sf.pooling(spkTm6, 3, 1, 1),
                                            sf.pooling(spkTmY5a, 3, 1, 1)],
                                      dim=1))
        spkT3, potT3 = sf.fire(potT3, self.convT3_t, True)
        if max_layer == 'T3':
            pot = sf.pointwise_inhibition(potT3)
            spk = pot.sign()
            winners = sf.get_k_winners(pot, spikes=spk)
            self.ctx["input_spikes"] = torch.cat([sf.pooling(spkL2, 3, 1, 1),
                                            sf.pooling(spkL4, 3, 1, 1),
                                            sf.pooling(spkL5, 3, 1, 1),
                                            sf.pooling(spkC2, 3, 1, 1),
                                            sf.pooling(spkC3, 3, 1, 1),
                                            sf.pooling(spkMi1, 3, 1, 1),
                                            sf.pooling(spkMi9, 3, 1, 1),
                                            sf.pooling(spkTm1, 3, 1, 1),
                                            sf.pooling(spkTm2, 3, 1, 1),
                                            sf.pooling(spkTm3, 3, 1, 1),
                                            sf.pooling(spkTm6, 3, 1, 1),
                                            sf.pooling(spkTmY5a, 3, 1, 1)],
                                      dim=1)
            self.ctx["potentials"] = pot
            self.ctx["output_spikes"] = spk
            self.ctx["winners"] = winners
            return spk, pot

        # LC4, LC17------------------------------------------------------------------------------------------------
        # LC4: Tm1, Tm2, Tm3, Tm4, Tm6, Tm9, TmY5a, T2, T2a, T3 - 5 window
        potLC4 = self.convLC4(torch.cat([sf.pooling(spkTm1, 5, 1, 2),
                                            sf.pooling(spkTm2, 5, 1, 2),
                                            sf.pooling(spkTm3, 5, 1, 2),
                                            sf.pooling(spkTm4, 5, 1, 2),
                                            sf.pooling(spkTm6, 5, 1, 2),
                                            sf.pooling(spkTm9, 5, 1, 2),
                                            sf.pooling(spkTmY5a, 5, 1, 2),
                                            sf.pooling(spkT2, 5, 1, 2),
                                            sf.pooling(spkT2a, 5, 1, 2),
                                            sf.pooling(spkT3, 5, 1, 2)],
                                      dim=1))
        spkLC4, potLC4 = sf.fire(potLC4, self.convLC4_t, True)
        if max_layer == 'LC4':
            pot = sf.pointwise_inhibition(potLC4)
            spk = pot.sign()
            winners = sf.get_k_winners(pot, spikes=spk)
            self.ctx["input_spikes"] = torch.cat([sf.pooling(spkTm1, 5, 1, 2),
                                            sf.pooling(spkTm2, 5, 1, 2),
                                            sf.pooling(spkTm3, 5, 1, 2),
                                            sf.pooling(spkTm4, 5, 1, 2),
                                            sf.pooling(spkTm6, 5, 1, 2),
                                            sf.pooling(spkTm9, 5, 1, 2),
                                            sf.pooling(spkTmY5a, 5, 1, 2),
                                            sf.pooling(spkT2, 5, 1, 2),
                                            sf.pooling(spkT2a, 5, 1, 2),
                                            sf.pooling(spkT3, 5, 1, 2)],
                                      dim=1)
            self.ctx["potentials"] = pot
            self.ctx["output_spikes"] = spk
            self.ctx["winners"] = winners
            return spk, pot
        
        
        # LC17: Tm2, Tm3, Tm4, Tm6, Tm9, Tm20, TmY5a, T2, T2a, T3 - 3 window
        potLC17 = self.convLC17(torch.cat([sf.pooling(spkTm2, 3, 1, 1),
                                            sf.pooling(spkTm3, 3, 1, 1),
                                            sf.pooling(spkTm4, 3, 1, 1),
                                            sf.pooling(spkTm6, 3, 1, 1),
                                            sf.pooling(spkTm9, 3, 1, 1),
                                            sf.pooling(spkTm20, 3, 1, 1),
                                            sf.pooling(spkTmY5a, 3, 1, 1),
                                            sf.pooling(spkT2, 3, 1, 1),
                                            sf.pooling(spkT2a, 3, 1, 1),
                                            sf.pooling(spkT3, 3, 1, 1)],
                                      dim=1))
        spkLC17, potLC17 = sf.fire(potLC17, self.convLC17_t, True)
        if max_layer == 'LC17':
            pot = sf.pointwise_inhibition(potLC17)
            spk = pot.sign()
            winners = sf.get_k_winners(pot, spikes=spk)
            self.ctx["input_spikes"] = torch.cat([sf.pooling(spkTm2, 3, 1, 1),
                                            sf.pooling(spkTm3, 3, 1, 1),
                                            sf.pooling(spkTm4, 3, 1, 1),
                                            sf.pooling(spkTm6, 3, 1, 1),
                                            sf.pooling(spkTm9, 3, 1, 1),
                                            sf.pooling(spkTm20, 3, 1, 1),
                                            sf.pooling(spkTmY5a, 3, 1, 1),
                                            sf.pooling(spkT2, 3, 1, 1),
                                            sf.pooling(spkT2a, 3, 1, 1),
                                            sf.pooling(spkT3, 3, 1, 1)],
                                      dim=1)
            self.ctx["potentials"] = pot
            self.ctx["output_spikes"] = spk
            self.ctx["winners"] = winners
            return spk, pot

        # central brain---------------------------------------------------------------------------------------------
        potCB = self.convCB(torch.cat([spkLC4, spkLC17],
                                      dim=1))
        spkCB, potCB = sf.fire(potCB)

        
        
        #------------------------------------------------------------------------------------------------------------
        if self.training:    
            # finish training
            winners = sf.get_k_winners(potCB, 1, 0, spkCB)
            self.ctx["input_spikes"] = torch.cat([spkLC4, spkLC17],
                                          dim=1)
            self.ctx["potentials"] = potCB
            self.ctx["output_spikes"] = spkCB
            self.ctx["winners"] = winners
            output = -1
            if len(winners) != 0:
                output = self.decision_map[winners[0][0]]
            return outputs
        else:
            winners = sf.get_k_winners(potCB, 1, 0, spkCB)
            output = -1
            if len(winners) != 0:
                output = self.decision_map[winners[0][0]]
            return output
        
        
        
        
        
        
        
    # STDP for layer layer_idx
    def stdp(self, layer_idx):
        
        if layer_idx == 'R7':
            self.stdpR7(self.ctx["input_spikes"], self.ctx["potentials"], self.ctx["output_spikes"], self.ctx["winners"])
        if layer_idx == 'R8':
            self.stdpR8(self.ctx["input_spikes"], self.ctx["potentials"], self.ctx["output_spikes"], self.ctx["winners"])
       
        if layer_idx == 'L1':
            self.stdpL1(self.ctx["input_spikes"], self.ctx["potentials"], self.ctx["output_spikes"], self.ctx["winners"])
        if layer_idx == 'L2':
            self.stdpL2(self.ctx["input_spikes"], self.ctx["potentials"], self.ctx["output_spikes"], self.ctx["winners"])
        if layer_idx == 'L3':
            self.stdpL3(sezf.ctx["input_spikes"], self.ctx["potentials"], self.ctx["output_spikes"], self.ctx["winners"])
        if layer_idx == 'L4':
            self.stdpL4(self.ctx["input_spikes"], self.ctx["potentials"], self.ctx["output_spikes"], self.ctx["winners"])
        if layer_idx == 'L5':
            self.stdpL5(self.ctx["input_spikes"], self.ctx["potentials"], self.ctx["output_spikes"], self.ctx["winners"])
        
        if layer_idx == 'C2':
            self.stdpC2(self.ctx["input_spikes"], self.ctx["potentials"], self.ctx["output_spikes"], self.ctx["winners"])
        if layer_idx == 'C3':
            self.stdpC3(self.ctx["input_spikes"], self.ctx["potentials"], self.ctx["output_spikes"], self.ctx["winners"])
        
        if layer_idx == 'Mi1':
            self.stdpMi1(self.ctx["input_spikes"], self.ctx["potentials"], self.ctx["output_spikes"], self.ctx["winners"])
        if layer_idx == 'Mi4':
            self.stdpMi4(self.ctx["input_spikes"], self.ctx["potentials"], self.ctx["output_spikes"], self.ctx["winners"])
        if layer_idx == 'Mi9':
            self.stdpMi9(self.ctx["input_spikes"], self.ctx["potentials"], self.ctx["output_spikes"], self.ctx["winners"])
        if layer_idx == 'Mi15':
            self.stdpMi15(self.ctx["input_spikes"], self.ctx["potentials"], self.ctx["output_spikes"], self.ctx["winners"])
        
        if layer_idx == 'Tm1':
            self.stdpTm1(self.ctx["input_spikes"], self.ctx["potentials"], self.ctx["output_spikes"], self.ctx["winners"])
        if layer_idx == 'Tm2':
            self.stdpTm2(self.ctx["input_spikes"], self.ctx["potentials"], self.ctx["output_spikes"], self.ctx["winners"])
        if layer_idx == 'Tm3':
            self.stdpTm3(self.ctx["input_spikes"], self.ctx["potentials"], self.ctx["output_spikes"], self.ctx["winners"])
        if layer_idx == 'Tm4':
            self.stdpTm4(self.ctx["input_spikes"], self.ctx["potentials"], self.ctx["output_spikes"], self.ctx["winners"])
        if layer_idx == 'Tm6':
            self.stdpTm6(self.ctx["input_spikes"], self.ctx["potentials"], self.ctx["output_spikes"], self.ctx["winners"])
        if layer_idx == 'Tm9':
            self.stdpTm9(self.ctx["input_spikes"], self.ctx["potentials"], self.ctx["output_spikes"], self.ctx["winners"])
        if layer_idx == 'Tm20':
            self.stdpTm20(self.ctx["input_spikes"], self.ctx["potentials"], self.ctx["output_spikes"], self.ctx["winners"])
        if layer_idx == 'TmY5a':
            self.stdpTmY5a(self.ctx["input_spikes"], self.ctx["potentials"], self.ctx["output_spikes"], self.ctx["winners"])
            
        if layer_idx == 'T2':
            self.stdpT2(self.ctx["input_spikes"], self.ctx["potentials"], self.ctx["output_spikes"], self.ctx["winners"])
        if layer_idx == 'T2a':
            self.stdpT2a(self.ctx["input_spikes"], self.ctx["potentials"], self.ctx["output_spikes"], self.ctx["winners"])
        if layer_idx == 'T3':
            self.stdpT3(self.ctx["input_spikes"], self.ctx["potentials"], self.ctx["output_spikes"], self.ctx["winners"])
        
        if layer_idx == 'LC4':
            self.stdpLC4(self.ctx["input_spikes"], self.ctx["potentials"], self.ctx["output_spikes"], self.ctx["winners"])
        if layer_idx == 'LC17':
            self.stdpLC17(self.ctx["input_spikes"], self.ctx["potentials"], self.ctx["output_spikes"], self.ctx["winners"])
       

    # learning rates updating
    def update_learning_rates(self, stdp_ap, stdp_an, anti_stdp_ap, anti_stdp_an):
        self.stdp_central_brain.update_all_learning_rate(stdp_ap, stdp_an)
        self.anti_stdp_central_brain.update_all_learning_rate(anti_stdp_an, anti_stdp_ap)

    # reward signal for ultimate layer
    def reward(self):
        self.stdp_central_brain(
            self.ctx["input_spikes"], self.ctx["potentials"], self.ctx["output_spikes"], self.ctx["winners"])
    
    # punishment signal for ultimate layer
    def punish(self):
        self.anti_stdp_central_brain(
            self.ctx["input_spikes"], self.ctx["potentials"], self.ctx["output_spikes"], self.ctx["winners"])

In [6]:
# training process for the layer layer_idx (R1_6, R7, R8, L1, L2, L3, .....)
def train_unsupervise(network, data, layer_idx):
    network.train()
    for i in range(len(data)):
        data_in = data[i]
        if use_cuda:
            data_in = data_in.cuda()
        network(data_in, layer_idx)
        network.stdp(layer_idx)

In [7]:
def train_rl(network, data, target):
    network.train()
    perf = np.array([0,0,0]) # correct, wrong, silence
    for i in range(len(data)):
        data_in = data[i]
        target_in = target[i]
        if use_cuda:
            data_in = data_in.cuda()
            target_in = target_in.cuda()
        # До последнего слоя
        d = network(data_in, 'central_brain')
        if d != -1:
            if d == target_in:
                perf[0]+=1
                network.reward()
            else:
                perf[1]+=1
                network.punish()
        else:
            perf[2]+=1
    return perf/len(data)

In [8]:
def test(network, data, target):
    network.eval()
    perf = np.array([0,0,0]) # correct, wrong, silence
    for i in range(len(data)):
        data_in = data[i]
        target_in = target[i]
        if use_cuda:
            data_in = data_in.cuda()
            target_in = target_in.cuda()
        d = network(data_in, 'central_brain')
        if d != -1:
            if d == target_in:
                perf[0]+=1
            else:
                perf[1]+=1
        else:
            perf[2]+=1
    return perf/len(data)

In [11]:
class Intensity2LatencyTransform:
    def __init__(self, filter, timesteps = 15):
        self.to_tensor = transforms.ToTensor()
        self.filter = filter
        self.temporal_transform = utils.Intensity2Latency(timesteps)
        self.cnt = 0
    def __call__(self, image):
        if self.cnt % 1000 == 0:
            print(self.cnt)
        self.cnt+=1
        image = self.to_tensor(image) * 255
        image.unsqueeze_(0)
        image = self.filter(image)
        image = sf.local_normalization(image, 8)
        temporal_image = self.temporal_transform(image)
        #return temporal_image.sign().byte()
        return temporal_image.sign()


In [ ]:
kernels = [ utils.DoGKernel(3,3/9,6/9),
            utils.DoGKernel(3,6/9,3/9),
            utils.DoGKernel(7,7/9,14/9),
            utils.DoGKernel(7,14/9,7/9),
            utils.DoGKernel(13,13/9,26/9),
            utils.DoGKernel(13,26/9,13/9)]
filter = utils.Filter(kernels, padding = 6, thresholds = 50)
trsf = Intensity2LatencyTransform(filter)

data_root = "data"
MNIST_train = utils.CacheDataset(torchvision.datasets.MNIST(root=data_root, train=True, download=True, transform = trsf))
MNIST_test = utils.CacheDataset(torchvision.datasets.MNIST(root=data_root, train=False, download=True, transform = trsf))
MNIST_loader = DataLoader(MNIST_train, batch_size=1000, shuffle=False)
MNIST_testLoader = DataLoader(MNIST_test, batch_size=len(MNIST_test), shuffle=False)

flyeye = FlyEye()
if use_cuda:
    flyeye.cuda()

    
#-------------------------------------------------------------------------------------------------------------------------
# Training R7
print("Training R7")
if os.path.isfile("saved_R7.net"):
    mozafari.load_state_dict(torch.load("saved_R7.net"))
else:
    for epoch in range(2):
        print("Epoch", epoch)
        iter = 0
        for data,targets in MNIST_loader:
            print("Iteration", iter)
            train_unsupervise(flyeye, data, 'R7')
            print("Done!")
            iter+=1
    torch.save(flyeye.state_dict(), "saved_R7.net")
    

# Training R8
print("Training R8")
if os.path.isfile("saved_R8.net"):
    mozafari.load_state_dict(torch.load("saved_R8.net"))
else:
    for epoch in range(2):
        print("Epoch", epoch)
        iter = 0
        for data,targets in MNIST_loader:
            print("Iteration", iter)
            train_unsupervise(flyeye, data, 'R8')
            print("Done!")
            iter+=1
    torch.save(flyeye.state_dict(), "saved_R8.net")
    
    
#-------------------------------------------------------------------------------------------------------------------------
# Training L1
print("Training L1")
if os.path.isfile("saved_L1.net"):
    mozafari.load_state_dict(torch.load("saved_L1.net"))
else:
    for epoch in range(2):
        print("Epoch", epoch)
        iter = 0
        for data,targets in MNIST_loader:
            print("Iteration", iter)
            train_unsupervise(flyeye, data, 'L1')
            print("Done!")
            iter+=1
    torch.save(flyeye.state_dict(), "saved_L1.net")
    
    
# Training L2
print("Training L2")
if os.path.isfile("saved_L2.net"):
    mozafari.load_state_dict(torch.load("saved_L2.net"))
else:
    for epoch in range(2):
        print("Epoch", epoch)
        iter = 0
        for data,targets in MNIST_loader:
            print("Iteration", iter)
            train_unsupervise(flyeye, data, 'L2')
            print("Done!")
            iter+=1
    torch.save(flyeye.state_dict(), "saved_L2.net")
    
    
# Training L3
print("Training L3")
if os.path.isfile("saved_L3.net"):
    mozafari.load_state_dict(torch.load("saved_L3.net"))
else:
    for epoch in range(2):
        print("Epoch", epoch)
        iter = 0
        for data,targets in MNIST_loader:
            print("Iteration", iter)
            train_unsupervise(flyeye, data, 'L3')
            print("Done!")
            iter+=1
    torch.save(flyeye.state_dict(), "saved_L3.net")

# Training L4
print("Training L4")
if os.path.isfile("saved_L4.net"):
    mozafari.load_state_dict(torch.load("saved_L4.net"))
else:
    for epoch in range(2):
        print("Epoch", epoch)
        iter = 0
        for data,targets in MNIST_loader:
            print("Iteration", iter)
            train_unsupervise(flyeye, data, 'L4')
            print("Done!")
            iter+=1
    torch.save(flyeye.state_dict(), "saved_L4.net")

    
# Training L5
print("Training L5")
if os.path.isfile("saved_L5.net"):
    mozafari.load_state_dict(torch.load("saved_L5.net"))
else:
    for epoch in range(2):
        print("Epoch", epoch)
        iter = 0
        for data,targets in MNIST_loader:
            print("Iteration", iter)
            train_unsupervise(flyeye, data, 'L5')
            print("Done!")
            iter+=1
    torch.save(flyeye.state_dict(), "saved_L5.net")
    
    
#-------------------------------------------------------------------------------------------------------------------------
# Training C2
print("Training C2")
if os.path.isfile("saved_C2.net"):
    mozafari.load_state_dict(torch.load("saved_C2.net"))
else:
    for epoch in range(2):
        print("Epoch", epoch)
        iter = 0
        for data,targets in MNIST_loader:
            print("Iteration", iter)
            train_unsupervise(flyeye, data, 'C2')
            print("Done!")
            iter+=1
    torch.save(flyeye.state_dict(), "saved_C2.net")
    
    
# Training C3
print("Training C3")
if os.path.isfile("saved_C3.net"):
    mozafari.load_state_dict(torch.load("saved_C3.net"))
else:
    for epoch in range(2):
        print("Epoch", epoch)
        iter = 0
        for data,targets in MNIST_loader:
            print("Iteration", iter)
            train_unsupervise(flyeye, data, 'C3')
            print("Done!")
            iter+=1
    torch.save(flyeye.state_dict(), "saved_C3.net")
    
    


#-------------------------------------------------------------------------------------------------------------------------
# Training Mi1
print("Training Mi1")
if os.path.isfile("saved_Mi1.net"):
    mozafari.load_state_dict(torch.load("saved_Mi1.net"))
else:
    for epoch in range(2):
        print("Epoch", epoch)
        iter = 0
        for data,targets in MNIST_loader:
            print("Iteration", iter)
            train_unsupervise(flyeye, data, 'Mi1')
            print("Done!")
            iter+=1
    torch.save(flyeye.state_dict(), "saved_Mi1.net")
    
    
    
# Training Mi4
print("Training Mi4")
if os.path.isfile("saved_Mi4.net"):
    mozafari.load_state_dict(torch.load("saved_Mi4.net"))
else:
    for epoch in range(2):
        print("Epoch", epoch)
        iter = 0
        for data,targets in MNIST_loader:
            print("Iteration", iter)
            train_unsupervise(flyeye, data, 'Mi4')
            print("Done!")
            iter+=1
    torch.save(flyeye.state_dict(), "saved_Mi4.net")
    
    
    
    
# Training Mi9
print("Training Mi9")
if os.path.isfile("saved_Mi9.net"):
    mozafari.load_state_dict(torch.load("saved_Mi9.net"))
else:
    for epoch in range(2):
        print("Epoch", epoch)
        iter = 0
        for data,targets in MNIST_loader:
            print("Iteration", iter)
            train_unsupervise(flyeye, data, 'Mi9')
            print("Done!")
            iter+=1
    torch.save(flyeye.state_dict(), "saved_Mi9.net")
    
    
    
    
# Training Mi15
print("Training Mi15")
if os.path.isfile("saved_Mi15.net"):
    mozafari.load_state_dict(torch.load("saved_Mi15.net"))
else:
    for epoch in range(2):
        print("Epoch", epoch)
        iter = 0
        for data,targets in MNIST_loader:
            print("Iteration", iter)
            train_unsupervise(flyeye, data, 'Mi15')
            print("Done!")
            iter+=1
    torch.save(flyeye.state_dict(), "saved_Mi15.net")

    
#-------------------------------------------------------------------------------------------------------------------------
# Training Tm1
print("Training Tm1")
if os.path.isfile("saved_Tm1.net"):
    mozafari.load_state_dict(torch.load("saved_Tm1.net"))
else:
    for epoch in range(2):
        print("Epoch", epoch)
        iter = 0
        for data,targets in MNIST_loader:
            print("Iteration", iter)
            train_unsupervise(flyeye, data, 'Tm1')
            print("Done!")
            iter+=1
    torch.save(flyeye.state_dict(), "saved_Tm1.net")
    
# Training Tm2
print("Training Tm2")
if os.path.isfile("saved_Tm2.net"):
    mozafari.load_state_dict(torch.load("saved_Tm2.net"))
else:
    for epoch in range(2):
        print("Epoch", epoch)
        iter = 0
        for data,targets in MNIST_loader:
            print("Iteration", iter)
            train_unsupervise(flyeye, data, 'Tm2')
            print("Done!")
            iter+=1
    torch.save(flyeye.state_dict(), "saved_Tm2.net")
    
# Training Tm3
print("Training Tm3")
if os.path.isfile("saved_Tm3.net"):
    mozafari.load_state_dict(torch.load("saved_Tm3.net"))
else:
    for epoch in range(2):
        print("Epoch", epoch)
        iter = 0
        for data,targets in MNIST_loader:
            print("Iteration", iter)
            train_unsupervise(flyeye, data, 'Tm3')
            print("Done!")
            iter+=1
    torch.save(flyeye.state_dict(), "saved_Tm3.net")
    
# Training Tm4
print("Training Tm4")
if os.path.isfile("saved_Tm4.net"):
    mozafari.load_state_dict(torch.load("saved_Tm4.net"))
else:
    for epoch in range(2):
        print("Epoch", epoch)
        iter = 0
        for data,targets in MNIST_loader:
            print("Iteration", iter)
            train_unsupervise(flyeye, data, 'Tm4')
            print("Done!")
            iter+=1
    torch.save(flyeye.state_dict(), "saved_Tm4.net")
    
# Training Tm6
print("Training Tm6")
if os.path.isfile("saved_Tm6.net"):
    mozafari.load_state_dict(torch.load("saved_Tm6.net"))
else:
    for epoch in range(2):
        print("Epoch", epoch)
        iter = 0
        for data,targets in MNIST_loader:
            print("Iteration", iter)
            train_unsupervise(flyeye, data, 'Tm6')
            print("Done!")
            iter+=1
    torch.save(flyeye.state_dict(), "saved_Tm6.net")
    
    
# Training Tm9
print("Training Tm9")
if os.path.isfile("saved_Tm9.net"):
    mozafari.load_state_dict(torch.load("saved_Tm9.net"))
else:
    for epoch in range(2):
        print("Epoch", epoch)
        iter = 0
        for data,targets in MNIST_loader:
            print("Iteration", iter)
            train_unsupervise(flyeye, data, 'Tm9')
            print("Done!")
            iter+=1
    torch.save(flyeye.state_dict(), "saved_Tm9.net")
    
# Training Tm20
print("Training Tm20")
if os.path.isfile("saved_Tm20.net"):
    mozafari.load_state_dict(torch.load("saved_Tm20.net"))
else:
    for epoch in range(2):
        print("Epoch", epoch)
        iter = 0
        for data,targets in MNIST_loader:
            print("Iteration", iter)
            train_unsupervise(flyeye, data, 'Tm20')
            print("Done!")
            iter+=1
    torch.save(flyeye.state_dict(), "saved_Tm20.net")
    
# Training TmY5a
print("Training TmY5a")
if os.path.isfile("saved_TmY5a.net"):
    mozafari.load_state_dict(torch.load("saved_TmY5a.net"))
else:
    for epoch in range(2):
        print("Epoch", epoch)
        iter = 0
        for data,targets in MNIST_loader:
            print("Iteration", iter)
            train_unsupervise(flyeye, data, 'TmY5a')
            print("Done!")
            iter+=1
    torch.save(flyeye.state_dict(), "saved_TmY5a.net")
    
    
    
    
#-------------------------------------------------------------------------------------------------------------------------- 
# Training T2
print("Training T2")
if os.path.isfile("saved_T2.net"):
    mozafari.load_state_dict(torch.load("saved_T2.net"))
else:
    for epoch in range(2):
        print("Epoch", epoch)
        iter = 0
        for data,targets in MNIST_loader:
            print("Iteration", iter)
            train_unsupervise(flyeye, data, 'T2')
            print("Done!")
            iter+=1
    torch.save(flyeye.state_dict(), "saved_T2.net")

# Training T2a  
print("Training T2a")
if os.path.isfile("saved_T2a.net"):
    mozafari.load_state_dict(torch.load("saved_T2a.net"))
else:
    for epoch in range(2):
        print("Epoch", epoch)
        iter = 0
        for data,targets in MNIST_loader:
            print("Iteration", iter)
            train_unsupervise(flyeye, data, 'T2a')
            print("Done!")
            iter+=1
    torch.save(flyeye.state_dict(), "saved_T2a.net")
    
# Training T3 
print("Training T3")
if os.path.isfile("saved_T3.net"):
    mozafari.load_state_dict(torch.load("saved_T3.net"))
else:
    for epoch in range(2):
        print("Epoch", epoch)
        iter = 0
        for data,targets in MNIST_loader:
            print("Iteration", iter)
            train_unsupervise(flyeye, data, 'T3')
            print("Done!")
            iter+=1
    torch.save(flyeye.state_dict(), "saved_T3.net")


    
#-------------------------------------------------------------------------------------------------------------------------- 
# Training LC4
print("Training LC4")
if os.path.isfile("saved_LC4.net"):
    mozafari.load_state_dict(torch.load("saved_LC4.net"))
else:
    for epoch in range(2):
        print("Epoch", epoch)
        iter = 0
        for data,targets in MNIST_loader:
            print("Iteration", iter)
            train_unsupervise(flyeye, data, 'LC4')
            print("Done!")
            iter+=1
    torch.save(flyeye.state_dict(), "saved_LC4.net")

# Training LC17  
print("Training LC17")
if os.path.isfile("saved_LC17.net"):
    mozafari.load_state_dict(torch.load("saved_LC17.net"))
else:
    for epoch in range(2):
        print("Epoch", epoch)
        iter = 0
        for data,targets in MNIST_loader:
            print("Iteration", iter)
            train_unsupervise(flyeye, data, 'LC17')
            print("Done!")
            iter+=1
    torch.save(flyeye.state_dict(), "saved_LC17.net")

    
    
    
    
#-------------------------------------------------------------------------------------------------------------------------- 

# initial adaptive learning rates

apr = mozafari.stdp_central_brain.learning_rate[0][0].item()
anr = mozafari.stdp_central_brain.learning_rate[0][1].item()
app = mozafari.anti_stdp_central_brain.learning_rate[0][1].item()
anp = mozafari.anti_stdp_central_brain.learning_rate[0][0].item()

adaptive_min = 0
adaptive_int = 1
apr_adapt = ((1.0 - 1.0 / 10) * adaptive_int + adaptive_min) * apr
anr_adapt = ((1.0 - 1.0 / 10) * adaptive_int + adaptive_min) * anr
app_adapt = ((1.0 / 10) * adaptive_int + adaptive_min) * app
anp_adapt = ((1.0 / 10) * adaptive_int + adaptive_min) * anp

# perf
best_train = np.array([0.0,0.0,0.0,0.0]) # correct, wrong, silence, epoch
best_test = np.array([0.0,0.0,0.0,0.0]) # correct, wrong, silence, epoch

# Training Central Brain
print("Training Central Brain")
for epoch in range(10):
    print("Epoch #:", epoch)
    perf_train = np.array([0.0,0.0,0.0])
    for data,targets in MNIST_loader:
        perf_train_batch = train_rl(flyeye, data, targets)
        print(perf_train_batch)
        #update adaptive learning rates
        apr_adapt = apr * (perf_train_batch[1] * adaptive_int + adaptive_min)
        anr_adapt = anr * (perf_train_batch[1] * adaptive_int + adaptive_min)
        app_adapt = app * (perf_train_batch[0] * adaptive_int + adaptive_min)
        anp_adapt = anp * (perf_train_batch[0] * adaptive_int + adaptive_min)
        flyeye.update_learning_rates(apr_adapt, anr_adapt, app_adapt, anp_adapt)
        perf_train += perf_train_batch
    perf_train /= len(MNIST_loader)
    if best_train[0] <= perf_train[0]:
        best_train = np.append(perf_train, epoch)
    print("Current Train:", perf_train)
    print("   Best Train:", best_train)

    for data,targets in MNIST_testLoader:
        perf_test = test(flyeye, data, targets)
        if best_test[0] <= perf_test[0]:
            best_test = np.append(perf_test, epoch)
            torch.save(flyeye.state_dict(), "saved.net")
        print(" Current Test:", perf_test)
        print("    Best Test:", best_test)

Training R7
Epoch 0
0
Iteration 0
Done!
1000
Iteration 1
Done!
2000
Iteration 2
Done!
3000
Iteration 3
Done!
4000
Iteration 4
Done!
5000
Iteration 5
Done!
6000
Iteration 6
Done!
7000
Iteration 7
Done!
8000
Iteration 8
Done!
9000
Iteration 9
Done!
10000
Iteration 10
Done!
11000
Iteration 11
Done!
12000
Iteration 12
Done!
13000
Iteration 13
Done!
14000
Iteration 14
Done!
15000
Iteration 15
Done!
16000
Iteration 16
Done!
17000
Iteration 17
Done!
18000
Iteration 18
Done!
19000
Iteration 19
Done!
20000
Iteration 20
Done!
21000
Iteration 21
Done!
22000
Iteration 22
Done!
23000
Iteration 23
Done!
24000
Iteration 24
Done!
25000
Iteration 25
Done!
26000
Iteration 26
Done!
27000
Iteration 27
Done!
28000
Iteration 28
Done!
29000
Iteration 29
Done!
30000
Iteration 30
Done!
31000
Iteration 31
Done!
32000
Iteration 32
Done!
33000
Iteration 33
Done!
34000
Iteration 34
Done!
35000
Iteration 35
Done!
36000
Iteration 36
Done!
37000
Iteration 37
Done!
38000
Iteration 38
Done!
39000
Iteration 39
Done!
400